In [1]:
import pandas as pd
import numpy as np

# 读取 npz 文件
data = np.load('./data/SWAT/train_swat.npz')
# ['train_x', 'train_target', 'val_x', 'val_target', 'mean', 'std']
data.files
# 获取 train_x 数组并查看其形状
# train_x = data['train_x']
# npy数据格式是一个四维的数组[N，H，W, C]，其中N代表数据集的总数，H, W，C分别代表每一张图片对应的长、宽、以及通道数。
# # Original train_x shape: (4195, 51, 1, 30)
# # print("Original train_x shape:", train_x.shape)


ModuleNotFoundError: No module named 'pandas'

In [ ]:
from lib.utils import *
from config_files.SWAT_config_gcn import Config
config = Config()

# 从配置中获取参数
device = config.device  # 设备
batch_size = config.batch_size  # 批处理大小
learning_rate = config.learning_rate  # 学习率
epochs = config.epochs  # 训练周期数
train_filename = config.train_filename  # 训练数据文件名
train_loader, train_target_tensor, val_loader, val_target_tensor, _mean, _std = load_data_train(train_filename, device, batch_size)


In [ ]:
mean = data['mean']  # 获取均值
mean

In [ ]:
std = data['std']  # 获取标准差
std

In [ ]:
import pandas as pd

In [ ]:
swat_normal = pd.read_csv("data/SWAT/SWaT_Dataset_Normal_v1.csv")
swat_normal['Normal/Attack'].unique()

In [ ]:
swat_abnormal = pd.read_csv("data/SWAT/SWaT_Dataset_Attack_v0.csv")  # 读取异常的SWaT数据集
# 新增一列 'Attack_Flag'，如果是 'Attack' 或 'A ttack' 则为 1，否则为 0
swat_abnormal['Attack_Flag'] = swat_abnormal['Normal/Attack'].apply(lambda x: 1 if x in ['Attack', 'A ttack'] else 0)

In [ ]:
import pandas as pd
import numpy as np
from config_files.SWAT_config import Config
# 初始化配置对象，读取配置参数
config = Config()
import copy

import os

current_dir = os.getcwd()
data_file = "data/SWAT/"
data_path = os.path.join(data_file, 'all_data_diffpool.npz')

data_path

'data/SWAT/all_data_diffpool.npz'

In [ ]:


# 读取异常的SWaT数据集
swat_abnormal = pd.read_csv("data/SWAT/SWaT_Dataset_Attack_v0.csv")

# 新增一列 'Attack_Flag'，如果是 'Attack' 或 'A ttack' 则为 1，否则为 0
swat_abnormal['Attack_Flag'] = swat_abnormal['Normal/Attack'].apply(lambda x: 1 if x in ['Attack', 'A ttack'] else 0)

# 获取 'Normal/Attack' 列的索引
col_index = swat_abnormal.columns.get_loc('Normal/Attack')

# 将 'Attack_Flag' 插入到 'Normal/Attack' 列之前
swat_abnormal.insert(col_index, 'Attack_Flag', swat_abnormal.pop('Attack_Flag'))
swat_abnormal_np = np.array(swat_abnormal.iloc[:, 1: -1])
swat_abnormal_np

array([[  2.427057, 522.8467  ,   2.      , ...,   1.      ,   1.      ,
          0.      ],
       [  2.446274, 522.886   ,   2.      , ...,   1.      ,   1.      ,
          0.      ],
       [  2.489191, 522.8467  ,   2.      , ...,   1.      ,   1.      ,
          0.      ],
       ...,
       [  2.531467, 520.6878  ,   2.      , ...,   1.      ,   1.      ,
          0.      ],
       [  2.521218, 520.7271  ,   2.      , ...,   1.      ,   1.      ,
          0.      ],
       [  2.501681, 521.1196  ,   2.      , ...,   1.      ,   1.      ,
          0.      ]])

In [ ]:
# 查看数组的形状
print(swat_abnormal_np.shape)

# 查看最后一列的数据
last_column = swat_abnormal_np[:, -1]
print(last_column)


(449919, 52)
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
def downsampling(mat, interval):
    """
    对矩阵进行降采样,根据指定的间隔,将矩阵的行数缩减为每interval取一行。
    :param mat: 输入的二维矩阵
    :param interval: 采样间隔
    :return: 降采样后的矩阵
    """
    num_row, num_col = mat.shape  # 获取矩阵的行数和列数
    res = num_row % interval  # 计算矩阵行数对采样间隔的余数
    if res != 0:  # 如果行数不能被采样间隔整除
        add_num = interval - res  # 计算需要补充的行数
        add_mat = np.zeros((add_num, num_col))  # 创建一个全零的矩阵进行补充
        mat = np.concatenate((mat, add_mat))  # 将原矩阵与补充的全零矩阵拼接在一起
    num_row, num_col = mat.shape  # 获取补充后的矩阵的行数和列数
    mat_tmp = np.zeros((interval, int(num_row / interval), num_col))  # 创建一个临时矩阵用于存储降采样数据
    for i in range(interval):  # 遍历每个采样间隔
        mat_tmp[i, ...] = mat[i::interval, :]  # 每隔interval行采样一次,存储到临时矩阵中
    return np.mean(mat_tmp, 0)  # 对每个采样间隔的数据取均值,返回降采样后的矩阵

In [ ]:
def max_min_norm(mat, max_=None, min_=None):
    """
    对矩阵进行最大-最小归一化,将数据缩放到[0,1]区间。
    :param mat: 输入矩阵
    :param max_: 每列的最大值（可选）
    :param min_: 每列的最小值（可选）
    :return: 归一化后的矩阵,以及最大值和最小值
    """
    if max_ is None:  # 如果没有提供最大值
        max_ = np.max(mat, 0)  # 计算每列的最大值
    if min_ is None:  # 如果没有提供最小值
        min_ = np.min(mat, 0)  # 计算每列的最小值
    nrow, ncol = mat.shape  # 获取矩阵的行数和列数
    for i in range(ncol):  # 遍历每一列
        if max_[i] == min_[i]:  # 如果最大值等于最小值
            mat[:, i] = mat[:, i] - min_[i]  # 该列的所有值减去最小值
        else:
            mat[:, i] = (mat[:, i] - min_[i]) / (max_[i] - min_[i])  # 归一化处理：减去最小值并除以范围
    return mat, max_, min_  # 返回归一化后的矩阵,最大值和最小值

In [ ]:
def swat_generate_data(xx, train_split, val_split, length, filename=None, max_=None, min_=None):
    # 对输入数据进行最大-最小归一化处理
    mat_, max_, min_ = max_min_norm(xx, max_, min_)
    nrow, ncol = xx.shape  # 获取数据的行数和列数
    
    # 按照划分比例分别获取训练集、验证集和测试集
    train_end = int(nrow * train_split)
    val_end = int(nrow * (train_split + val_split))
    
    xx_train = xx[:train_end, :]  # 训练集数据
    xx_val = xx[train_end:val_end, :]  # 验证集数据
    xx_test = xx[val_end:, :]  # 测试集数据

    # 生成训练集样本
    train_x = np.zeros((xx_train.shape[0] - length + 1, length, ncol))
    for i in range(train_x.shape[0]):
        train_x[i, ...] = xx_train[i:i + length, :]

    # 生成验证集样本
    valid_x = np.zeros((xx_val.shape[0] - length + 1, length, ncol))
    for i in range(valid_x.shape[0]):
        valid_x[i, ...] = xx_val[i:i + length, :]

    # 生成测试集样本
    test_x = np.zeros((xx_test.shape[0] - length + 1, length, ncol))
    for i in range(test_x.shape[0]):
        test_x[i, ...] = xx_test[i:i + length, :]

    # 构建数据字典
    all_data = {
        'train': {
            'x': np.expand_dims(np.transpose(train_x, (0, 2, 1)), 2),  # 训练集输入数据
            'target': np.expand_dims(np.transpose(train_x, (0, 2, 1)), 2),  # 训练集目标数据
        },
        'val': {
            'x': np.expand_dims(np.transpose(valid_x, (0, 2, 1)), 2),  # 验证集输入数据
            'target': np.expand_dims(np.transpose(valid_x, (0, 2, 1)), 2),  # 验证集目标数据
        },
        'test': {
            'x': np.expand_dims(np.transpose(test_x, (0, 2, 1)), 2),  # 测试集输入数据
            'target': np.expand_dims(np.transpose(test_x, (0, 2, 1)), 2),  # 测试集目标数据
        },
        'stats': {
            '_mean': np.zeros((1, 1, 3, 1)),  # 初始化均值
            '_std': np.zeros((1, 1, 3, 1)),  # 初始化标准差
        }
    }

    # 如果没有提供文件名，就直接返回数据
    if filename is None:
        return max_, min_, all_data

    # 将数据保存为.npz文件
    np.savez_compressed(filename,
                        train_x=all_data['train']['x'], train_target=all_data['train']['target'],
                        val_x=all_data['val']['x'], val_target=all_data['val']['target'],
                        test_x=all_data['test']['x'], test_target=all_data['test']['target'],
                        mean=all_data['stats']['_mean'], std=all_data['stats']['_std'])

    return max_, min_, all_data

split = 0.8  # 训练集和验证集的划分比例，80%数据用于训练
length = config.target_len  # 配置文件中指定的序列长度

data = downsampling(swat_abnormal_np, config.downsampling_fre)
max_, min_, all_data = swat_generate_data(copy.copy(data), train_split=0.7, val_split=0.2, length=50, filename=data_path)

In [ ]:
def load_data_train(filename, DEVICE, batch_size, shuffle=True):
    # 加载npz文件
    file_data = np.load(filename)

    # 加载训练数据和目标
    train_x = file_data['train_x']  # 训练集输入
    train_x = train_x[:, :, 0:1, :]  # (B, N, F, T)
    train_target = file_data['train_target']  # 训练集目标

    # 加载验证数据和目标
    val_x = file_data['val_x']  # 验证集输入
    val_x = val_x[:, :, 0:1, :]  # (B, N, F, T)
    val_target = file_data['val_target']  # 验证集目标

    # 加载测试数据和目标
    test_x = file_data['test_x']  # 测试集输入
    test_x = test_x[:, :, 0:1, :]  # (B, N, F, T)
    test_target = file_data['test_target']  # 测试集目标

    # 提取均值和标准差
    mean = file_data['mean'][:, :, 0:1, :]  # (1, 1, 3, 1)
    std = file_data['std'][:, :, 0:1, :]  # (1, 1, 3, 1)

    # -------- 处理训练集标签 --------
    train_label = train_target[:, :, -1]  # 提取最后一列作为标签
    train_x_tensor = torch.from_numpy(train_x).type(torch.FloatTensor).to(DEVICE)
    train_target_tensor = torch.from_numpy(train_target).type(torch.FloatTensor).to(DEVICE)
    train_label_tensor = torch.from_numpy(train_label).type(torch.LongTensor).to(DEVICE)

    train_dataset = torch.utils.data.TensorDataset(train_x_tensor, train_target_tensor, train_label_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

    # -------- 处理验证集标签 --------
    val_label = val_target[:, :, -1]  # 提取最后一列作为标签
    val_x_tensor = torch.from_numpy(val_x).type(torch.FloatTensor).to(DEVICE)
    val_target_tensor = torch.from_numpy(val_target).type(torch.FloatTensor).to(DEVICE)
    val_label_tensor = torch.from_numpy(val_label).type(torch.LongTensor).to(DEVICE)

    val_dataset = torch.utils.data.TensorDataset(val_x_tensor, val_target_tensor, val_label_tensor)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # -------- 处理测试集标签 --------
    test_label = test_target[:, :, -1]  # 提取最后一列作为标签
    test_x_tensor = torch.from_numpy(test_x).type(torch.FloatTensor).to(DEVICE)
    test_target_tensor = torch.from_numpy(test_target).type(torch.FloatTensor).to(DEVICE)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor).to(DEVICE)

    test_dataset = torch.utils.data.TensorDataset(test_x_tensor, test_target_tensor, test_label_tensor)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # 打印数据维度信息
    print('train:', train_x_tensor.size(), train_target_tensor.size(), train_label_tensor.size())
    print('val:', val_x_tensor.size(), val_target_tensor.size(), val_label_tensor.size())
    print('test:', test_x_tensor.size(), test_target_tensor.size(), test_label_tensor.size())


    return train_loader, train_target_tensor, val_loader, val_target_tensor, test_loader, test_target_tensor, mean, std


In [ ]:
import torch


from config_files.SWAT_config import Config
# 初始化配置对象，读取配置参数
config = Config()

# 从配置中获取参数
device = config.device  # 设备
batch_size = config.batch_size  # 批处理大小
learning_rate = config.learning_rate  # 学习率
epochs = config.epochs  # 训练周期数
train_filename = config.train_filename  # 训练数据文件名

# 文件路径和参数
filename = "data/SWAT/all_data_diffpool.npz"
shuffle = True

# 调用 load_data_train 函数，获取加载的数据
train_loader, train_target_tensor, val_loader, val_target_tensor, test_loader, test_target_tensor, mean, std = load_data_train(
    filename=filename,
    DEVICE=device,
    batch_size=batch_size,
    shuffle=shuffle
)

# 打印数据维度信息
print(f"训练集目标维度: {train_target_tensor.size()}")
print(f"验证集目标维度: {val_target_tensor.size()}")
print(f"测试集目标维度: {test_target_tensor.size()}")
print(f"均值维度: {mean.shape}")
print(f"标准差维度: {std.shape}")

# 示例：遍历训练集的数据
for batch_x, batch_y, batch_label in train_loader:
    print(f"输入数据 (batch_x) 维度: {batch_x.size()}")
    print(f"目标数据 (batch_y) 维度: {batch_y.size()}")
    print(f"标签 (batch_label) 维度: {batch_label.size()}")
    print(batch_label.unique())
    # break  # 仅打印第一个batch的维度信息


train: torch.Size([4470, 52, 1, 30]) torch.Size([4470, 52, 1, 30]) torch.Size([4470, 52, 30])
val: torch.Size([1471, 52, 1, 30]) torch.Size([1471, 52, 1, 30]) torch.Size([1471, 52, 30])
test: torch.Size([1471, 52, 1, 30]) torch.Size([1471, 52, 1, 30]) torch.Size([1471, 52, 30])
训练集目标维度: torch.Size([4470, 52, 1, 30])
验证集目标维度: torch.Size([1471, 52, 1, 30])
测试集目标维度: torch.Size([1471, 52, 1, 30])
均值维度: (1, 1, 1, 1)
标准差维度: (1, 1, 1, 1)
输入数据 (batch_x) 维度: torch.Size([128, 52, 1, 30])
目标数据 (batch_y) 维度: torch.Size([128, 52, 1, 30])
标签 (batch_label) 维度: torch.Size([128, 52, 30])
tensor([0, 1], device='cuda:0')
输入数据 (batch_x) 维度: torch.Size([128, 52, 1, 30])
目标数据 (batch_y) 维度: torch.Size([128, 52, 1, 30])
标签 (batch_label) 维度: torch.Size([128, 52, 30])
tensor([0, 1], device='cuda:0')
输入数据 (batch_x) 维度: torch.Size([128, 52, 1, 30])
目标数据 (batch_y) 维度: torch.Size([128, 52, 1, 30])
标签 (batch_label) 维度: torch.Size([128, 52, 30])
tensor([0, 1], device='cuda:0')
输入数据 (batch_x) 维度: torch.Size([128, 52, 